In [9]:
from typing import List
from dotenv import load_dotenv
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1 as discoveryengine

# TODO(developer): Uncomment these variables before running the sample.
project_id = "nthurc-aisearch-202406"
location = "global"                    # Values: "global", "us", "eu"
data_store_id = "pdf-data-search_1718003912950"
search_queries = ["我想申請社宅", "三峽國光"]
load_dotenv()


def multi_turn_search_sample(
    project_id: str,
    location: str,
    data_store_id: str,
    search_queries: List[str],
) -> List[discoveryengine.ConverseConversationResponse]:
    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    client_options = (
        ClientOptions(
            api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.ConversationalSearchServiceClient(
        client_options=client_options
    )

    # Initialize Multi-Turn Session
    conversation = client.create_conversation(
        # The full resource name of the data store
        # e.g. projects/{project_id}/locations/{location}/dataStores/{data_store_id}
        parent=client.data_store_path(
            project=project_id, location=location, data_store=data_store_id
        ),
        conversation=discoveryengine.Conversation(),
    )

    for search_query in search_queries:
        # Add new message to session
        request = discoveryengine.ConverseConversationRequest(
            name=conversation.name,
            query=discoveryengine.TextInput(input=search_query),
            serving_config=client.serving_config_path(
                project=project_id,
                location=location,
                data_store=data_store_id,
                serving_config="default_config",
            ),
            # Options for the returned summary
            summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
                # Number of results to include in summary
                summary_result_count=3,
                include_citations=True,
            ),
        )
        response = client.converse_conversation(request)

        print(f"Reply: {response.reply.summary.summary_text}\n")

        for i, result in enumerate(response.search_results, 1):
            result_data = result.document.derived_struct_data
            print(result_data)
            print(f"[{i}]")
            print(f"Link: {result_data['link']}")
            print(f"First Snippet: {result_data['snippets'][0]['snippet']}")
            print(
                "First Extractive Answer: \n"
                f"\tPage: {result_data['extractive_answers'][0]['pageNumber']}\n"
                f"\tContent: {result_data['extractive_answers'][0]['content']}\n\n"
            )
        print("\n\n")

        return response


response = multi_turn_search_sample(
    project_id, location, data_store_id, search_queries)

Reply: 要申请新北市青年社会住宅，您需要满足一些条件，例如年满18岁或已婚未满18岁，在新北市设籍、就学或就业，且家庭成员在台北市、新北市、桃园市和基隆市均无自有住宅 [3]。此外，家庭成员的收入也需要符合当年度的公告标准 [3]。您需要准备一些文件，例如全户最新电子户籍謄本正本或全户户口名簿影本，以及最近一年的财产归属资料清单正本 [1]。您可以在新北市住房及都市更新中心网站上在线申请，也可以到土城员和青年社会住宅管理中心现场申请 [2, 3]。 


[1]
Link: gs://nthurc-aisearch-poc-data/PDF_PoC/申請須知(上).pdf
First Snippet: 本次開放<b>申請</b>「土城大安、土城永和、板橋江翠、土城明德 2 號、五股成州、新店央北及三峽國 光」等 7 處<b>社宅</b>，<b>申請</b>人應就所<b>申請</b>承租<b>社宅</b>，分別填寫<b>申請</b>書，並依<b>申請</b>身分&nbsp;...
First Extractive Answer: 
	Page: 6
	Content: 6 (五) 人口數限制： 1. 家庭成員2 人以上（含2 人）始可<b>申請</b>2 房型。 2. 家庭成員3 人以上（含3 人）始可<b>申請</b>3 房型。 (六) 住宅資源不得重複享有限制： 1. 家庭成員有<b>申請</b>等候承租國民住宅、公共（營）住宅或社會住宅之情事，於簽訂本案租賃契約前， 應事先完成放棄國民住宅之等候。 2. 家庭成員現已承租國民住宅、公共（營）住宅或社會住宅者，於簽訂本案租賃契約前，應事先完成退租國民住宅、公共（營）住宅或社會住宅及完成清空點交與遷出程序。 三、<b>申請</b>人應以書面檢附下列文件： (一)113 年度第1 次隨到隨辦-青年社會住宅<b>申請</b>書正本。 本次開放<b>申請</b>「土城大安、土城永和、板橋江翠、土城明德2 號、五股成州、新店央北及三峽國光」等7 處<b>社宅</b>，<b>申請</b>人應就所<b>申請</b>承租<b>社宅</b>，分別填寫<b>申請</b>書，並依<b>申請</b>身分類別及資格，檢附(二)至(七)相關文件。 (二)全戶最新電子戶籍謄本正本或全戶戶口名簿影本1.檢附【民國113 年2 月29 日(含

In [4]:
# code heavily inspired by https://cloud.google.com/generative-ai-app-builder/docs/multi-turn-search#genappbuilder_multi_turn_search-python
from typing import List
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1 as discoveryengine
import os
import argparse  # Import the argparse module


def multi_turn_search_sample(
    project_id: str,
    location: str,
    data_store_id: str,
    search_queries: List[str],
    conversation_id: str = None,
) -> List[discoveryengine.ConverseConversationResponse]:
    client_options = (
        ClientOptions(
            api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.ConversationalSearchServiceClient(
        client_options=client_options
    )

    if conversation_id:
        conversation_name = client.conversation_path(
            project=project_id, location=location, data_store=data_store_id, conversation=conversation_id)
    else:
        conversation = client.create_conversation(
            parent=client.data_store_path(
                project=project_id, location=location, data_store=data_store_id),
            conversation=discoveryengine.Conversation(),
        )
        conversation_name = conversation.name
        print(f"Conversation ID: {conversation.name.split('/')[-1]}")

    for search_query in search_queries:
        request = discoveryengine.ConverseConversationRequest(
            name=conversation_name,
            query=discoveryengine.TextInput(input=search_query),
            serving_config=client.serving_config_path(
                project=project_id,
                location=location,
                data_store=data_store_id,
                serving_config="default_config",
            ),
            summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
                summary_result_count=3,
                include_citations=True,
            ),
        )
        response = client.converse_conversation(request)
        print(response)


search_queries = ["戶口名簿是什麼？"]
multi_turn_search_sample(project_id, location, data_store_id,
                         search_queries, "8354159113724969335")

reply {
  summary {
    summary_text: "戶口名簿是用来证明您和您的家庭成员的户籍关系的官方文件 [1, 2, 3]。 您可以选择提供全户最新电子户籍謄本正本或户口名簿影本 [1, 2, 3]。 申请人需要在申请期限内备妥相关文件，并于收件时间亲至土城员和青年社会住宅管理中心完成送件 [2]。 申请日期应为民国113年2月29日(含)以后 [1]。 此外，您还需要提供新北市设籍或就学、就業證明正本 [1]。 \n"
    summary_with_metadata {
      summary: "戶口名簿是用来证明您和您的家庭成员的户籍关系的官方文件。 您可以选择提供全户最新电子户籍謄本正本或户口名簿影本。 申请人需要在申请期限内备妥相关文件，并于收件时间亲至土城员和青年社会住宅管理中心完成送件。 申请日期应为民国113年2月29日(含)以后。 此外，您还需要提供新北市设籍或就学、就業證明正本。 \n"
      citation_metadata {
        citations {
          end_index: 84
          sources {
          }
          sources {
            reference_index: 1
          }
          sources {
            reference_index: 2
          }
        }
        citations {
          start_index: 85
          end_index: 166
          sources {
          }
          sources {
            reference_index: 1
          }
          sources {
            reference_index: 2
          }
        }
        citations {
          start_index: 167
          end_index: 302
          sources {
  

In [20]:
import urllib
caption = response.reply.summary.summary_with_metadata.references

for i, reference in enumerate(caption, 1):

    print(f"[{i}] : [{reference.title}]({reference.uri})")

extractive_answers = []
caption = response.reply.summary.summary_with_metadata.references
for i, reference in enumerate(caption):
    if reference.uri.startswith('gs://'):
        link = 'https://storage.cloud.google.com/' + \
            urllib.parse.quote(reference.uri.replace(  # type: ignore
                'gs://', ''))  # type: ignore
    else:
        link = ""
    ref = f"[{i}] : [{reference.title}]({link})"
    extractive_answers.append(ref)
print(extractive_answers)

[1] : [申請須知(上)](gs://nthurc-aisearch-poc-data/PDF_PoC/申請須知(上).pdf)
[2] : [申請須知 (下)](gs://nthurc-aisearch-poc-data/PDF_PoC/申請須知 (下).pdf)
[3] : [網站常見問題集](gs://nthurc-aisearch-poc-data/PDF_PoC/網站常見問題集.pdf)
['[0] : [申請須知(上)](https://storage.cloud.google.com/nthurc-aisearch-poc-data/PDF_PoC/%E7%94%B3%E8%AB%8B%E9%A0%88%E7%9F%A5%28%E4%B8%8A%29.pdf)', '[1] : [申請須知 (下)](https://storage.cloud.google.com/nthurc-aisearch-poc-data/PDF_PoC/%E7%94%B3%E8%AB%8B%E9%A0%88%E7%9F%A5%20%28%E4%B8%8B%29.pdf)', '[2] : [網站常見問題集](https://storage.cloud.google.com/nthurc-aisearch-poc-data/PDF_PoC/%E7%B6%B2%E7%AB%99%E5%B8%B8%E8%A6%8B%E5%95%8F%E9%A1%8C%E9%9B%86.pdf)']
